In [197]:
# from pyspark.sql import SparkSession

In [198]:
# spark = (
#     SparkSession.builder
#     .appName("SparkETLExample")
#     .getOrCreate())

In [199]:
# "first_name	last_name	salary	dept_name	salary_increment".replace("	",",")

In [200]:
# flat_schema = "first_name STRING, last_name STRING, salary INT, \ndept_name STRING, salary_increment float"
# flat_data = "flat_data.csv"

In [201]:
# flat_file = spark.read.format("csv").schema(flat_schema).option("header","true").load(flat_data)

In [202]:
# #kya e lazmi ha k new file me dpartmane unique hongy?
# #kya e lazmi ha k new file me name unique hongy?
# unique_dept = flat_file.select("dept_name","salary_increment").distinct()
# unique_dept.show()

In [203]:
# unique_dept.createOrReplaceTempView("newDeptData")

In [204]:
# spark.sql("SELECT * FROM newDeptData").show()

In [205]:
# spark = SparkSession \
#         .builder \
#         .appName("Python Spark SQL data source example") \
#         .getOrCreate()

In [206]:
# df = (
#     spark
#     .read 
    
#     .jdbc("jdbc:postgresql://localhost:5432/etlcode", "com_dim_city",properties={"user": "postgres", "password": "usman123"})
# )

# df.createOrReplaceTempView("[your_table]")

# sqlDF = spark.sql("SELECT * FROM [your_table] LIMIT 10")
# sqlDF.show()

In [226]:
from pyspark.sql import SparkSession

# the Spark session should be instantiated as follows
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.jars", "/home/usman/Downloads/postgresql-42.2.19.jar") \
    .getOrCreate()

In [227]:
flat_schema = "first_name STRING, last_name STRING, salary INT, \ndept_name STRING, salary_increment float"
flat_data = "flat_data.csv"

In [228]:
flat_file = spark.read.format("csv").schema(flat_schema).option("header","true").load(flat_data)

In [229]:
#kya e lazmi ha k new file me dpartmane unique hongy?
#kya e lazmi ha k new file me name unique hongy?
unique_dept = flat_file.select("dept_name","salary_increment").distinct()
unique_dept.show()

+---------+----------------+
|dept_name|salary_increment|
+---------+----------------+
|Marketing|            16.0|
|    Sales|            17.0|
|       IT|            15.0|
|  Finance|            10.0|
+---------+----------------+



In [230]:
url='jdbc:postgresql://localhost:5432/etlcode' # jdbc:postgresql://<host>:<port>/<database>
dbtable='department'
user='postgres'
password="usman123"
driver='org.postgresql.Driver'

In [231]:
jdbcDF = spark.read.format("jdbc"). \
options(
         url= url,
         dbtable='department',
         user=user,
         password=password,
         driver=driver
).load()

In [232]:
jdbcDF.show()

+--------------------+---------+--------------------+
|                  id|     name|    salary_increment|
+--------------------+---------+--------------------+
|ec09fd05-b2d4-4b1...|    Sales|10.00000000000000...|
|f1cdd777-357b-47d...|Marketing|16.00000000000000...|
|4e46d780-cf04-486...|  Finance|10.00000000000000...|
|ba570a7f-f189-46c...|       IT|15.00000000000000...|
+--------------------+---------+--------------------+



In [189]:

# df = (
#     spark.
#     read.
#     jdbc(url = "jdbc:postgresql://localhost:5432/database_example",\
#          table = "(SELECT Employee.id, Employee.name AS employee_name, Employee.salary, Employee.departmentid, Department.name AS department_name \
#                      FROM Employee INNER JOIN Department ON Employee.DepartmentId = Department.ID) AS my_table",
#                      properties={"user": "postgres", "password": "1234", "driver": 'org.postgresql.Driver'}).createTempView('tbl')
#      )

In [190]:
#depart = (jdbcDF.join(unique_dept,jdbcDF.name != unique_dept.dept_name, how="right"))

In [191]:
#depart.show()

In [192]:
unique_dept.createOrReplaceTempView("newDeptData")

In [193]:
jdbcDF.createOrReplaceTempView("oldDeptData")

In [194]:
spark.sql("SELECT * FROM newDeptData").show()

+---------+----------------+
|dept_name|salary_increment|
+---------+----------------+
|Marketing|            16.0|
|    Sales|            17.0|
|       IT|            15.0|
|  Finance|            10.0|
+---------+----------------+



In [195]:
spark.sql("SELECT * FROM oldDeptData").show()

+--------------------+---------+--------------------+
|                  id|     name|    salary_increment|
+--------------------+---------+--------------------+
|ec09fd05-b2d4-4b1...|    Sales|10.00000000000000...|
|f1cdd777-357b-47d...|Marketing|16.00000000000000...|
|4e46d780-cf04-486...|  Finance|10.00000000000000...|
|ba570a7f-f189-46c...|       IT|15.00000000000000...|
+--------------------+---------+--------------------+



In [244]:
spark.read.format("jdbc"). \
options(
         url= url,
        query = "UPDATE department SET name = 'S' WHERE name = 'Sales' ",
         user=user,
         password=password,
         driver=driver
)

In [245]:
jdbcDF.show()

+--------------------+---------+--------------------+
|                  id|     name|    salary_increment|
+--------------------+---------+--------------------+
|ec09fd05-b2d4-4b1...|    Sales|10.00000000000000...|
|f1cdd777-357b-47d...|Marketing|16.00000000000000...|
|4e46d780-cf04-486...|  Finance|10.00000000000000...|
|ba570a7f-f189-46c...|       IT|15.00000000000000...|
+--------------------+---------+--------------------+



In [196]:
#1st 
#update existing depart's salary_increment
spark.sql("""
SELECT odd.salary_increment,ndd.salary_increment FROM newDeptData as ndd
INNER JOIN oldDeptData odd on ndd.dept_name = odd.name

""").show()

+--------------------+----------------+
|    salary_increment|salary_increment|
+--------------------+----------------+
|10.00000000000000...|            17.0|
|16.00000000000000...|            16.0|
|10.00000000000000...|            10.0|
|15.00000000000000...|            15.0|
+--------------------+----------------+



In [162]:
#2.
# insert new depart
dept_col = spark.sql("""
SELECT ndp.dept_name as name, ndp.salary_increment FROM newDeptData as ndp
LEFT JOIN oldDeptData odd on ndp.dept_name = odd.name
where odd.name IS NULL

""")

In [163]:
dept_col.show()

+----+----------------+
|name|salary_increment|
+----+----------------+
+----+----------------+



In [164]:
url='jdbc:postgresql://localhost:5432/etlcode' # jdbc:postgresql://<host>:<port>/<database>
dbtable='department'
user='postgres'
password="usman123"
driver='org.postgresql.Driver'

In [165]:
prop={"user":user,
        "password":password,
     "driver":"org.postgresql.Driver"
     }

In [166]:
#dept_col.write.jdbc(url='jdbc:postgresql://localhost:5432/etlcode', mode="append",table='department', properties=prop)

In [167]:
dept_col.write.jdbc(url="jdbc:postgresql://localhost:5432/etlcode", table=dbtable, mode="append", properties=prop)

In [169]:
dept_col.show()

+----+----------------+
|name|salary_increment|
+----+----------------+
+----+----------------+



In [170]:
jdbcDF.show()

+--------------------+---------+--------------------+
|                  id|     name|    salary_increment|
+--------------------+---------+--------------------+
|ec09fd05-b2d4-4b1...|    Sales|10.00000000000000...|
|f1cdd777-357b-47d...|Marketing|16.00000000000000...|
|4e46d780-cf04-486...|  Finance|10.00000000000000...|
|ba570a7f-f189-46c...|       IT|15.00000000000000...|
+--------------------+---------+--------------------+



In [174]:
old_emp_data = spark.read.format("jdbc"). \
options(
         url= url,
         dbtable="employee",
         user=user,
         password=password,
         driver=driver
).load()

In [176]:
old_emp_data.show()

+---+----------+---------+------+-------------+
| id|first_name|last_name|salary|department_id|
+---+----------+---------+------+-------------+
+---+----------+---------+------+-------------+



In [177]:
new_emp_data = flat_file.select('first_name','last_name','salary')

In [178]:
old_emp_data.createOrReplaceTempView('old_emp_data')
new_emp_data.createOrReplaceTempView('new_emp_data')

In [181]:
spark.sql("""
SELECT oed.salary as old,ned.salary as new from old_emp_data oed
INNER JOIN new_emp_data ned on oed.first_name = ned.first_name

""").show()

+---+---+
|old|new|
+---+---+
+---+---+



In [250]:
# File upload
# Fetch Unique Dept
# Join get new
# Insert new
# Get All Dept UUId
# Map All UUID to df
# Department updated
# Department Inserted
# Department Fetched with uuid
# Employee MApped wiht department UUID
# Employee Updated
# Employeee INserted
# Calculations